In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np
import csv
import sympy
from numpy import array

### Mediator Parameters

$n$ : the number of times a jobs is executed for verification.

$PR \in [0,1] \subset \mathbb{R}$ : the penalty rate, used to determine the minimum D of both Job Creators and Resource Providers

$NDPR \in [0,1] \subset \mathbb{R}$ : the non-deterministic penalty rate, used to determine the Job Creators non-deterministic D

$baseFee = JCmediatorPrice + RPmediatorPrice$ : The fee the Mediator requires to be available for a mediation. A mediator can be used if `available < JCmediatorPrice + RPmediatorPrice`

$mediationFee = D-pi$ : The amount Mediator receives for mediating a given dispute. 

$ND\_Fee = NDPR \times pi-pi$ : The amount Mediator receives for mediating a given dispute and finding JC guilty of non-determinism. 

### Mediator Model

In [2]:
# Mediator Parameters
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='PR', continuous_update=False) #Penalty Rate
wNDPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='NDPR',continuous_update=False)#Non-deterministic Penalty rate
wp_ETarget = widgets.FloatSlider(min=0,max=1,step=.01,value=1, description='p_E',continuous_update=False)#p_E target slider

def MNDU(JCmediatorPrice, RPmediatorPrice):
    baseFee = JCmediatorPrice + RPmediatorPrice
    mediationFee = p_v*(p2+(1-p_E))*(p1**n)*(D-pi)
    ND_Fee = p_v*(p2+(1-p_E))*(1-(p1**n))*(NDD-pi) 
    return available + mediationFee + ND_Fee




### Solver Parameters

$matchFee$ = The amount a solver receives for finding a given match. 

### Solver Model

In [3]:
#Solver
def SNDU(): #solver nondeterminisitc utility
    matchFee = RPMatchingFee + JCMatchingFee
    return matchFee

# JC Model

### Job Creator parameters

$\pi = instructionCount*JO.rate + bandwidthUsage*JO.rate$ : The max pi the JC is willing to pay to have a job executed.

$p_{v}$ : probability the JC verifies result a1. 

$p_{va2}$ : probability the JC verifies result a2. This probability is 0 because it can just go straight to mediation. 

$C_v$ : cost to JC to verify result.

$B$ : the value or B JC gets when a job is performed correctly.

$matchPrice$ : the amount JC pays to Solver for finding a suitable RP. 

$mediatorPrice$ : The amount JC pays to mediator for being available. 

$p_1$ : probability that job returns answer in class $a1$

$p_2 = 1-p_1$ : probability that job returns answer in class $a2$

$D = \pi \cdot (PR + n)$ : the amount the Job Creator must post as a security D in order to get matched. It multiplied by the $n$, which as a reminder is the number of times the mediator will replicate a job. 

This model takes into account when the JC is attempting to gain utility though tricking the mediator into believing a RP did a job incorrectly and giving it compensation. The JC can trick the mediator by providing a job with a non-deterministic output. The output is in one of two answer classes, $a1$ or $a2$ with $P(a1)>>P(a2)$. If RP responds with $a1$ the JC will pay RP. If RP responds with $a2$ JC will request mediation, expecting the mediator to find the answer to be $a1$ and determine that the $RP$ must have cheated. 
We only need to considert two answer classes because when the Mediator is performing verification it assigns fault to the RP only if it finds there to be one answer which does not match the RP answer, if it finds two answers that differ from one another, regardless of if they match the RP answer, it faults the JC. This means that JC wants to maximize the chance that the Mediator will get the same answer reapeatedly which is not Bed by having additional answer classes.  

We need to make sure that the platform compensation model does not make this strategy feasible. 

**Assumptions**

* The job has probability p1 of resulting in an answer in class a1 and 1-p1 of resulting in class a2

In [4]:
# JC parameters
wpi = widgets.IntSlider(min=0,max=1000,step=1,value=1, continuous_update=False)
C_r = 0
wp_v= widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='p_v', continuous_update=False) # how often JC verifies
wC_v= widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='C_v',continuous_update=False) # cost for JC to verify
wB=widgets.IntSlider(min=0,max=1000,step=1,value=2, continuous_update=False)
matchPrice = 1
mediatorPrice = 1

def JCU(n,PR,NDPR, p1,B,pi,p_v=1,C_v=0, p_E=1):
    p2 = 1-p1
    
    NDD = pi * NDPR + (pi* n)
     
    JC_HRP  =  p_v*p_E*p1*(B-pi-C_v) # RP correctly (p_E) returns answer 1 (p1) and pay up to max_pi (-pi).
    
    NDJC_HRP_Gain = p_v*p_E*p2*(p1**n)*(B + pi -C_v) # receives gas pi because it had to spend some to request mediation.
    NDJC_HRP_Lose = p_v*p_E*p2*(1-(p1**n))*(B-NDD-C_r-C_v) #RP returns answer 2 correctly or responds incorrectly (p2+(1-p_E)) and send to mediator who finds out nonDeterminism (1-p1^n), pay fine and reward (f2+r)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose
    
    DRP_Gain = p_v*(1-p_E)*(p1**n)*(pi - C_v) 
    DRP_Lose = p_v*(1-p_E)*(1-p1**n)*(-NDD-C_r-C_v)# compute incorrectly (1-p_E) get sent to mediation, finds nondeterminism (1-p1^n) get compensation.
#     DRP_scared = p_v*(1-p_E)*(-pi-C_v) # JC doesn't want to risk getting caught, just pay RP
#     DRP =  max(DRP_Gain+DRP_Lose, DRP_scared)
    DRP = DRP_Gain + DRP_Lose
    
    
    noID_HRP = (1-p_v)*p_E*(B-pi)
    noID_DRP = (1-p_v)*(1-p_E)*(-pi)
    noVerifyCost = noID_HRP + noID_DRP
    
    JCnonDet = JC_HRP + ND + DRP + noVerifyCost
    return JCnonDet


### Upper Bounds on Job Creator Cheating

\begin{equation}
    \begin{alignedat}{3}
        % HJC_HRP
        \textbf{JCU} = & p_v\cdot p_E\cdot p_1 \quad &&  && \cdot (B-\pi-C_{v}) \\
        % ND
        %  NDJC_HRP Gain  
        +& p_v\cdot p_E\cdot (1-p_1)  && \cdot (p_1^{n})  && \cdot (B + \pi -C_{v}) \\
        %  NDJC_HRP Loss  
        +& p_v\cdot p_E\cdot (1-p_1)  && \cdot (1-p_1^{n})  && \cdot (B-D-C_r-C_{v}) \\
        % DRP
        % Gain
        +& p_v\cdot (1-p_E) && \cdot (p_1^{n})  && \cdot (\pi - C_{v}) \\
        % Loss
        +& p_v\cdot (1-p_E) && \cdot (1-p_1^{n}) && \cdot (-D-C_r-C_{v}) \\
        % No verify
        +& (1-p_v)\cdot p_E &&  && \cdot (B-\pi) \\
        +& (1-p_v)\cdot (1-p_E)  &&  && \cdot (-\pi) \\
    \end{alignedat}
    \label{eq:JCU}
\end{equation}

In [5]:
sn, sPR, sNDPR, sp1, sB, spi, sp_v, sC_v, sp_E, sC_r= sympy.symbols('n PR NDPR p1 B pi p_v C_v p_E C_r', real=True)

JU = JCU(sn, sPR, sNDPR, sp1, sB, spi, sp_v, sC_v, sp_E)
display(JU)


p1*p_E*p_v*(B - C_v - pi) + p1**n*p_E*p_v*(1 - p1)*(B - C_v + pi) + p1**n*p_v*(1 - p_E)*(-C_v + pi) + p_E*p_v*(1 - p1)*(1 - p1**n)*(B - C_v - NDPR*pi - n*pi) + p_E*(1 - p_v)*(B - pi) + p_v*(1 - p1**n)*(1 - p_E)*(-C_v - NDPR*pi - n*pi) - pi*(1 - p_E)*(1 - p_v)

Assume worst case scenario

$p_v = 1$ ; $p_E = 1$ ; $C_v = 0$ ; $C_r = 0$

In [6]:
sJU = JU.subs({sp_v:1, sp_E:1, sC_v:0, sC_r:0 })
display(sJU)
sJU = sympy.expand(sJU)
# display(JU)
sJU = sympy.simplify(sJU)
# display(JU)
sJU = sympy.collect(sJU,sp1)
sJU = sympy.collect(sJU,sp1**(sn))
sJU = sympy.collect(sJU,sp1**(sn+1))
sJU = sympy.collect(sJU,spi)
display(sJU)


p1*(B - pi) + p1**n*(1 - p1)*(B + pi) + (1 - p1)*(1 - p1**n)*(B - NDPR*pi - n*pi)

B + pi*(-NDPR - n + p1*(NDPR + n - 1) + p1**n*(NDPR + n + 1) + p1**(n + 1)*(-NDPR - n - 1))

In [7]:
# Compare
n = 4
NDPR = 0
p = .5
b = 2
pi = 1

ySJ = JCU(n=n,PR=0,NDPR=NDPR,p1=p,B=b,pi=pi,p_v=1,C_v=0,p_E=1)
print(ySJ)

ysJU = sJU.subs({sn:n, sNDPR:NDPR, sp1:p, sB:b, spi:pi})
print(ysJU)

-0.34375
-0.343750000000000


In [8]:
dJU = sympy.diff(sJU,sp1)
# display(dJU)
dJU = sympy.collect(dJU,sp1)
display(dJU)

pi*(NDPR + n - 1 + (n*p1**n*(NDPR + n + 1) + p1**(n + 1)*(n + 1)*(-NDPR - n - 1))/p1)

In [9]:
sdJU = dJU.subs({sNDPR:0})
display(sdJU)

sdJU = sdJU * sp1/spi
display(sdJU)

sdJU = sympy.expand(sdJU)
display(sdJU)

sdJU = sympy.simplify(sdJU)
display(sdJU)

sdJU = sympy.collect(sdJU,sp1)
# display(sdJU)

sdJU = sympy.collect(sdJU,sp1**sn)
# display(sdJU)

sdJU = sympy.collect(sdJU,sp1**(sn+1))
display(sdJU)

# p1max = sympy.solveset(sdJU,sp1)
# display(p1max)


pi*(n - 1 + (n*p1**n*(n + 1) + p1**(n + 1)*(-n - 1)*(n + 1))/p1)

p1*(n - 1 + (n*p1**n*(n + 1) + p1**(n + 1)*(-n - 1)*(n + 1))/p1)

-n**2*p1*p1**n + n**2*p1**n - 2*n*p1*p1**n + n*p1 + n*p1**n - p1*p1**n - p1

n**2*p1**n - n**2*p1**(n + 1) + n*p1 + n*p1**n - 2*n*p1**(n + 1) - p1 - p1**(n + 1)

p1*(n - 1) + p1**n*(n**2 + n) + p1**(n + 1)*(-n**2 - 2*n - 1)

In [10]:
def plotdJU(n):
    
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
    ydu = list(map(lambda p: float(sdJU.subs({sn:n, sp1:p})), x))
    
    plt.plot(x,ydu, label="dU")
    
    plt.xlabel('p1')
    plt.ylabel('dUtil') 
    
    plt.legend()
    plt.show
    

In [11]:
# ui1 = widgets.HBox([wn,wPR])
# ui2 = widgets.HBox([wB,wpi])
# interactive_plot = widgets.interactive_output(plotdJU,{'n':wn});
# # output = interactive_plot.children[-1]
# # interactive_plot.layout.height = '400px'
# display(interactive_plot,ui1,ui2)

The Job Creators verification strategy is controlled by $p_v$, which is the probability of verifying a given result. 

The type of job, is captured by the $\pi$ and the $C_v$ terms. The price $\pi$ captures the complexity of execution, $C_v$ captures the complexity of verification.

A Resource Provider may choose to execute a job correctly or not. We capture this with the parameter $p_E$. If $p_E=1$ then the $RP$ executes all jobs correctly, if $p_E=0$ then it does not execute any jobs correctly.  

<!---While running non-deterministic jobs a Job Creator must decide whether the pay the $RP$ for a bad response, or risk getting caught in mediation. Currently the choice is made in the `max(RPfraud1,RPfraud2)` term. If the Job Creator is running deterministic jobs if it verifies and finds a bad result it will request mediation.--->

## Resource Providier

The Resource Provider only has one method to try to abuse the system. That is to return a result and hope that the Job Creator does not verify the job. If the Job Creator never verifies the Resource Provider will always cheat, and if it always verifies the Resource Provider will never cheat. However, the RP does not know how often the Job Creator will verify. 

### Resource Provider Model
The most RP can be paid for executing a job is $\pi = instructionCount*JO.rate + bandwidthUsage*JO.rate$. 

Jobs are only worth executing if the operating costs are covered by the Resource providers fees, which are at most $\pi$ for a given job. 

The Resource Provider does not only want to cover operating costs however, it additionally wants to make a profit. 

$profit = \pi-cost$

Since the price $\pi$ varies from job to job, the RP specifies a return on investment $roi$: 

$roi = \frac{\pi-cost}{\pi}$

For the model we need to estimate the cost of executing a job $cost$. We compute this with the given $\pi$ and required roi. This is reasonable since the JC specifies the $\pi$ and $\pi$ is ralated to job complexity, and the execution cost will increase with complexity.

$cost = \pi*(1-roi)$ 

The platform will only match a job to an RP if $\pi>requestedPrice$.

$requestedPrice$ covers the cost of execution. So in the analysis if $\pi<requestedPrice$ the point is infeasible. 

$requestedPrice$ is not used in the RP model, only computed. 

$requestedPrice = cost$

In a situation where the Job Creator is sending non-deterministic jobs, a Resource provider will lose 

$p_v \times (1-p_1) \times p_1^n\times D$

If the RP assumes $p_v=1$(JC always verifies), and that JC uses optimal $p_1$ given $n$ and $NDPR$ it can determine the maximum percent $I$ of its D that it will lose to the mediator being tricked. By adding $ I \times D$ to its $requestedPrice$ it can mitigate the damage done by cheating Job Creators. $I$ is an insurance percent against cheating. This results in a new $requestedPrice$ :  

$requestedPrice = \pi \times(1-roi) + D \times I $


When the RP is matched at its' $requestedPrice$ the $cost$ will effectively be reduced by $D \times I$ so the effective $cost$ used in the model is:

$cost = \pi*(1-roi) - D \times I $

In the `plot` function $requestedPrice$ is compared against $\pi$ to check if the point is valid. 


In [12]:

wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='p_E', continuous_update=False) # probability that RP runs correctly
wroi = widgets.FloatSlider(min=0,max=1,step=.01,value=.5,description='roi', continuous_update=False) # what percent of reward is profit
winsurance = widgets.FloatSlider(min=0,max=1,step=.01,value=0 ,description='I',continuous_update=False) # % of deposit that can be recovered because execution is so cheap.
wC_f= widgets.FloatSlider(min=0,max=1000,step=.1,value=0, description='C_f',continuous_update=False) # cost for RP to be fraudulant 

matchPrice = 1
mediatorPrice = 1


def RPU(n,PR,NDPR,p1,pi,p_v=1,C_f=0,p_E=1, ec=0, I=0):
    p2 = 1-p1
#     D = pi * PR
#     D = pi * PR + pi*n
    D = pi * PR + (pi * n)
    
    
    execCost = ec
    
    insurance =  I

    #Cj is cost of execution for fraudulant job, picking a random number or "close" value
    fraudCost = C_f
    
    JC_HRP = p_v*p_E*p1*(pi-execCost) #compute answer 1 (p1) correctly (p_E) receive reward (pi)
    
    NDJC_HRP_Gain = p_v*p_E*p2*(1-p1**n)*(pi-execCost) #compute answer 2 (p2) correctly (p_E) get sent to mediation, who finds nondeterminism (1-p1^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k
    NDJC_HRP_Lose = p_v*p_E*p2*(p1**n)*(-D-execCost) # compute answer 2 correctly (p2) get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose 
    
    DRP_Gain = p_v*(1-p_E)*(1-p1**n)*(pi-fraudCost)# compute incorrectly (1-p_E) get sent to mediation, finds nondeterminism (1-p1^n) get compensation.
    DRP_Lose = p_v*(1-p_E)*(p1**n)*(-D-fraudCost) #compute incorrectly (1-p_E) get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    DRP = DRP_Gain+DRP_Lose
    
#     DRP_scared  = p_v*(1-p_E)*(pi-fraudCost) # JC doesn't want to risk getting caught, just pay RP. 
    
    #     noVerifypayout = (1-p_v)*pi #compute answer 1 (p1) correctly (p_E) receive reward (pi)
    noID_HRP = (1-p_v) * p_E * (pi - execCost)
    noID_DRP = (1-p_v) * (1-p_E) * (pi - fraudCost)
    noID = noID_HRP + noID_DRP
    
    
    RCnonDet = JC_HRP + ND + DRP + insurance + noID
    return RCnonDet


In [13]:
points = 51
x,step = np.linspace(0,1,points,retstep=True)
print("step size: %s" %step)

def plot(n,PR,NDPR,pi,b,p_v=1,C_v=0, p_E=1,roi=1,Insurance=0,C_f=0):   
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
    yJ = JCU(n=n,PR=PR,NDPR=NDPR,p1=np.ones(points),B=b,pi=pi,p_v=p_v,C_v=C_v,p_E=p_E)
    yJND = JCU(n,PR,NDPR, p1=x,B=b,pi=pi,p_v=p_v,C_v=C_v, p_E=p_E)
    
    
    ySJ = JCU(n=n,PR=0,NDPR=NDPR,p1=x,B=b,pi=pi,p_v=1,C_v=0,p_E=1)
    
    ysJU = list(map(lambda p: float(sJU.subs({sn:n, sNDPR:0, sp1:p, sB:b, spi:pi})),x))
    ydJU  = list(map(lambda p: float(sdJU.subs({sn:n, sp1:p})), x))
    
    
    ec = pi*(1-roi)
    D = pi * PR + (pi * n)
    I = D*Insurance
    
    yRD = RPU(n,PR,NDPR,p1=x,pi=pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I)

#     yRD = list(map(lambda p1: RPU(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    
    plt.plot(x,yJ, label="J")
    plt.plot(x,yJND, label="JND")
    plt.plot(x,ySJ, label="SJ")
    
    plt.plot(x,ysJU, label="sJU")
    plt.plot(x,ydJU, label="dJU")
   
    plt.plot(x,yRD, label="RD")
    plt.xlabel('p1')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("n: %s" %n)
    print("Price: %s" %pi)
    D = pi * PR + pi*n
    print("D: %s" %D)
    
    maxJND = max(yJND)
    print("maxJND: %s" %maxJND)
    
    maxRD = max(yRD)
    print("maxRD: %s" %maxRD)
    
    
    minRD = min(yRD)
    print("minRD: %s" %minRD)
    
    
    p1maxJ = x[np.argmax(yJND)]
    p2 = 1 - p1maxJ
    print("P1 for JC max util: %s" %p1maxJ)
    print("")
    
    p1minRD = x[np.argmin(yRD)]
    print("P1 for RP min util: %s" %p1minRD)
    print("")
    
    
    print("Resource Provider")
    execCost = pi*(1-roi)
    requstedPrice = execCost + D*Insurance   
    print("requstedPrice: %s" %requstedPrice)   
    print("pi > requstedPrice, Matchable? : %s" %(pi>requstedPrice))
    
    
   

step size: 0.02


In [14]:

# platform
ui1 = widgets.HBox([wn,wPR,wNDPR]) 
# Job creator
ui21 = widgets.HBox([wB,wpi])
ui22 = widgets.HBox([wp_v,wC_v])
# Resource Provider
ui31 = widgets.HBox([wC_f,wp_E])
ui32 = widgets.HBox([wroi,winsurance])

# interactive_plot = widgets.interactive(plot, n=wn,PR=wPR,NDPR=wNDPR,pi=wpi,p_v=wp_v,cj=wcj, p_E=wp_E,roi=wroi,Insurance=winsurance);
interactive_plot = widgets.interactive_output(plot,{'n':wn,'PR':wPR,'NDPR':wNDPR,'pi':wpi,'b':wB,'p_v':wp_v,'C_v':wC_v, 'p_E':wp_E,'roi':wroi,'Insurance':winsurance,'C_f':wC_f });
# output = interactive_plot.children[-1]
# interactive_plot.layout.height = '900px'
display(interactive_plot,ui1,ui21,ui22,ui31,ui32)

Output()

# B Analysis

This section analyzes when the JC is sending real jobs that provide some benefit $B$ when they are successfully completed. The JC is attempting to increase its utility by adding a non-deterministic element to the jobs in an effort to have some jobs completed for free by accusing the RP of failing to execute them correctly. This could be done by changing the input so that some parts of the code are normally dorment and only activated when a certain data input is provided.

### Protocol Following model
If the JC follows the protocol its utility is the B, or value, it receives from the Job execution minus the amount it pays to the RP. This is assuming that the RP is playing by the rules of the game, otherwise the JC could have better utility since $\pi$ will be refunded and **B** will still be recieved since the mediator will return the result. 

This model is further simplified by the assumption that a Job can be checked for free, and that the JC can check the result of every job. This is a reasonable simplification since there are jobs whose verification cost is much less than the cost of execution, and we are interested in identifying the worst case scenario we need to protect against. 

In [15]:
def protocolU(B, pi):
    return B - pi

### Non-Deterministic Model
The key element of this model is the amount of utility the JC can get for **free** from the system. 
The free utility is the probability that the RP got result 2 multiplied by the probability that the mediator got result 1 $n$ times multiplied by the job B, minus the probability that the RP got result 2 multiplied by the probability that the mediator did not get result 1 for any of $n$ executions multiplied by the D, whose value is determined by the pi set by the JC multiplied by the non-deterministic penalty rate set by the Mediator. 

This model is the best the JC can hope for. The RP could fail to complete jobs, in which case the JC loses the B, and either pays $pi$ or ND D.

In [16]:
def NDU(n,NDPR, p1,pi,B):
    p2 = 1-p1
    NDD = pi * NDPR + (pi*n)
    paid = p1*(B-pi)
#     free = p2*((p1**n)*B - (1-p1**n)*NDD)
    free = p2*((p1**n)*(B+pi) + (1-p1**n)*(B-NDD))
    total = paid + free
    return total

# 𝑝1(𝐵−𝜋)+𝑝𝑛1(1−𝑝1)(𝐵+𝜋)+(1−𝑝1)(1−𝑝𝑛1)(𝐵−𝑁𝐷𝑃𝑅𝜋−𝑛𝜋)

    

### Evaluation

In [17]:
def plot2(n,NDPR, pi,B):
    points = 101
    x,step = np.linspace(0,1,points,retstep=True)
    
    yJ = JCU(n=n,PR=0,NDPR=NDPR,p1=np.ones(points),B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    ySJ = JCU(n=n,PR=0,NDPR=NDPR,p1=x,B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    
    yPU = list(map(lambda p1: protocolU(B,pi),x))
    yNDU = list(map(lambda p1: NDU(n,NDPR, p1,pi,B),x))
    
    
    plt.figure(3,figsize=(10,5))    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
    plt.plot(x,yPU, label="PU")
    plt.plot(x,yNDU, label="NDU")
    
    plt.plot(x,yJ, label="J")
    plt.plot(x,ySJ, label="SJ")
    
    
    
    
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("Protocol util: %s" %yPU[0])
    print("max ND util: %s" %max(yNDU))
    print("ND improvement over Protocol: %s" %(max(yNDU)-yPU[0]))    
    print("%% improvement : %s %%" %(100*abs((max(yNDU)-yPU[0])/yPU[0])))
    print("optimal P(p1): %s" %x[yNDU.index(max(yNDU))])
    
    with open('JCND_file.csv', mode='w') as JCND_file:
        JCwriter = csv.writer(JCND_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        JCwriter.writerow(['p1','JC Deterministic', 'JC Non-Deterministic'])
        
        for i in range(101):
            JCwriter.writerow([i/100-1/100, yPU[i], yNDU[i] ])

    
    

In [18]:
interactive_plot2 = widgets.interactive(plot2, n=wn,NDPR=wNDPR, pi=wpi,B=wB);
output = interactive_plot2.children[-1]
output.layout.height = '600px'
interactive_plot2

#n = 4
#NDPR = 24? if pi is included in free job.

interactive(children=(IntSlider(value=1, continuous_update=False, description='n', max=10), FloatSlider(value=…

### Simple Inep_Euality for PU<NDU


**Define Terms**

$NDU = paid + free$
                                 
$NDU = p1*(B-pi) + p2*((p1^{n})*(B+pi) - (1-p1^{n})*NDD) $
                                 
$NDU = p1*(B-pi) + p2*((p1^{n})*(B+pi) - (1-p1^{n})*(pi * M_{NDPR})) $



$PU = B - pi $

**Set Inequality**

$PU > NDU$

$ B - pi > p1*(B-pi) + p2*((p1^{n})*(B+pi) - (1-p1^{n})*(pi * M_{NDPR}))$

$ B + (1-p1^{n})*(pi * M_{NDPR})) - p1*(B-pi) - p2*((p1^{n})*(B+pi) - pi > 0$

When the above inep_Euality holds, then it is economically disadvantageous to pursue a nondeterministic strategy (NDU).

### Bounding the problem
$free = (1-p_v)P^nb - (1-p_v)(1-P^n)Rc$

**Derivative**

$\frac{d}{dP}(free) = \frac{d}{dP} (1-p_v)P^nb - (1-p_v)(1-P^n)Rc$

$0 = \frac{d}{dP} (P^nb - P^{n+1} - Rc + P^nRc + PRc - P^{n+1}Rc$)

$0 = nP^{n-1}b - (n+1)P^{n}b - 0 + nP^{n-1}Rc + Rc - (n+1)P^{n}Rc$

**Collect Terms**

$ nP^{n-1}b + nP^{n-1}Rc + Rc =  (n+1)P^{n}b + (n+1)P^{n}Rc$

$ nP^{n-1}(b+Rc) + Rc =  (n+1)P^{n}(b+Rc)$

$ nP^{n-1} + \frac{Rc}{b+Rc} =  (n+1)P^{n}$


$\frac{Rc}{b+Rc} + nP^{n-1} - (n+1)P^n = 0 $

**Let penalty rate R=0**

B and cost become irrelevant and we are left with:

$nP^{n-1}  = (n+1)P^n $

**Simplify**

$\frac{n}{n+1} = P$

So the maximum the JC can make with no penalty rate will be when $P = \frac{n}{n+1}$

$P$ is the liklihood of getting answer 1 and $n$ is the number of times the mediator verifies, so we can force the JC to cheat less and limit its additional utility. 

For example if $n=4$ then $P=.8$, so for optimal gains JC will make it so that a1 appears 80% of the time, anything else would give it less reward, so if we assume the worst case and handle it, anything less severe will be handled as well. 

**If R is not 0**

any change to $b$ or $c$ will never make lower probability of a1 more optimal. 

**For $b<c$ and R=1**

The normal protocol loses, but the ND protocol loses somewhat less until $b=.9c$ approximately, as which point ND can win slightly. 

It can win .694% utility units when $b=90$ and $c=100$.

**For $b=c$ and R=1**

Optimal is when probability of a1 is 90%. 
Normal protocal loses, and ND protocol wins 9.683% of benfit(or pi since they are ep_Eual) as utility. 

**For $b>c$ and R=1**

As $b$ grows the advantage over the protocol decreases compared to $b=c$

When $b=4.87c$ the added B is less than 1%. 

**Things get worse for JC if R increases.**
The worst case for the system is $b=c$ as the JC can get the most advantage. This can be mitigated by assuming JC is cheating and taking a 9.683% tax on the pi it includes, or increasing R to, apparently, 24, after which point ND can't win and is only better than protocol if $b=.762c$ or less (meaning its losing badly since it needed 90% to win in the first place).




# Deterministic Mixed Strategy


The deterministic mixed strategy will be defined by a variable p_E. p_E is the probability of the RP to cheat the job, and submit an unrelated junk answer to avoid computing the job.


**Variable**

JC Variables

$p$: probability JC verifies job after recieving solution

$r$: reward JC pays RP for computing job successfully

$b$: B gained from job

$c$: cost to compute job

$cv$: cost to verify job


RP Variables

$p_E$: probability to cheat

$cd$: cost of deceit. The cost to provide a fake resonse to a job. 


Platform variables

$m$: cost to mediate

$f$: penality for cheating


**Inep_Euality**

`RPmixed = p_Ep(r-c) + p_E(1-p_v)*(r-c) + (1-p_E)*(1-p_v)*(r-cd) -(1-p_E)*p_v*(f+m+cd)`

**Simplify**

`RPmixed = p_E*(r-c) + (1-p_E)*(-cd +(1-p_v)*r -p_v*(f+m) )`

`JCmixed = pp_E*(b-r-cv) + (1-p_v)*(p_E)(b-r) + p(1-p_E)(f-cv) + (1-p_v)(1-p_E)(-r)`

**Simplify**

`JCmixed = p_E(p(b-r-cv) + (1-p_v)(b-r)) + (1-p_E)(p(f-cv) + (1-p_v)(-r))`

`JCmixed = p_E(pb-pr-pcv + b-r -pb +pr) + (1-p_E)(p(f-cv) + (1-p_v)(-r))`

`JCmixed = p_E(-pcv + b -r) + (1-p_E)(p(f-cv) + (1-p_v)(-r))`

`JCmixed = p_E(b - r - p(cv)) + (1-p_E)(p(f-cv) + (1-p_v)(-r))`

`JCmixed = p_E(b - r - p(cv)) + (1-p_E)(p(r+f-cv) -r)`


RP decides to cheat when:

$r-c > ((1-p_v*pj) * r - p_v*pj*(f+m) -cd)  $

 $ r-c -(1-p_v*pj) * r +  p_v*pj*(f+m) +cd  > 0$  
 
 $ -(1-p_v*pj) * r +  p_v*pj*(f+m)  > -cd -r + c$
 
$  cd + p( f + m + r ) > c$

$p >\frac{c - cd}{f + m + r}$

If the above inep_Euality holds, then RP will be economically incentivized to always complete the job.


In [19]:
# Plotting Deterministic Mixed Strategy

def JCmixed(b,r,p_v,cj,f, c,cd,m,p_E):
    p_E=getp_E(c,cd,f,m,r,p_v)
    noCheat = p_E*(b-r - p_v*cj)  
    cheat = (1-p_E)*(p_v*(f-cj) + (1-p_v)*(-r))
    total=noCheat+cheat
    return total

def RPmixed(r,p_v,f,c, m, cd,p_E):
    p_E=getp_E(c,cd,f,m,r,p_v)
    noCheat = p_E*(r -c) 
    cheat = (1-p_E)*((1-p_v) * r - p_v*(f+m) -cd)  
    total= cheat + noCheat
    return total

def p_EValue(c,cd,f,m,r):
    return (c-cd)/(f+m+r)


def getp_E(c,cd,f,m,r,p_v):
    p_EPoint = (c-cd)/(f+m+r)
    if (p_v>p_EPoint):
        return(1)
    else:
        return(0)
    
    return(-1)


#Plot
def plot3( pi,B, computation, fine, cj, cd, m):

 
    r = pi#1.7
    f = fine#pi*2
    b = B#2
    c = computation#1
    cj = cj#c
    cd = cd#0
    m=m#c*10
    p_E=1
    
    
    print("p_E Change Point = \t",(c-cd)/(f+m+r))


    x,step = np.linspace(0,1,101,retstep=True)
    
    
    p_E = p_EValue(c,cd,f,m,r)
    
    yPU  = list(map(lambda p_v: JCmixed(B,pi,p_v,cj,f, c,cd,m,p_E),x))
    yDU  = list(map(lambda p_v: RPmixed(pi,p_v,f,c, m, cd,p_E),x))
    yNDU = list(map(lambda p_v: JCU(m,PR=2,NDPR=2, p1=1,B=B,pi=pi,p_v=p_v,C_v=cj, p_E=p_E),x))
    
    p_Eval = list(map(lambda p_v: getp_E(c,cd,f,m,r,p_v),x))
    
    plt.figure(3,figsize=(10,5))    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    plt.plot(x,yNDU, label="JCND")
    plt.plot(x,yPU, label="JC")
    plt.plot(x,yDU, label="RP")
    plt.plot(x,p_Eval,label="p_Eval")
    plt.xlabel('P (Probability of Verification)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show

In [20]:
#pi,B, computation, fine, cj, cd, m):
wComputation=widgets.IntSlider(min=0,max=100,step=0.01,value=4, continuous_update=False)
wFine=widgets.IntSlider(min=0,max=100000,step=0.01,value=60, continuous_update=False)
wcj=widgets.IntSlider(min=0,max=1000,step=0.01,value=4, continuous_update=False)
wcd=widgets.IntSlider(min=0,max=100,step=0.01,value=0, continuous_update=False)
wm=widgets.IntSlider(min=0,max=1000000,step=0.01,value=20, continuous_update=False)


interactive_plot3 = widgets.interactive(plot3,pi=wpi,B=wB, computation=wComputation, fine=wFine, cj=wcj, cd=wcd, m=wm);
output = interactive_plot3.children[-1]
# output.layout.height = '1000px'
interactive_plot3

interactive(children=(IntSlider(value=1, continuous_update=False, description='pi', max=1000), IntSlider(value…

In [21]:
def plotRP(n,PR,NDPR,p1,pi,b,p_v=1,C_v=0,roi=1,Insurance=0,C_f=0):   
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    print("Comparison of RPNDD with varying p_E")
    x,step = np.linspace(0,1,51,retstep=True)
    print("step size: %s" %step)
    
    ec = pi*(1-roi)
    D = pi * PR + (pi * n)
    I = D*Insurance
    
    yRNDD = list(map(lambda p_E: RPU(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    print("n,PR,NDPR,pi,p1,p_v=p_v,p_E=p_E,ec=ec,I=Insurance,C_f=C_f")
    print( n,PR,NDPR,pi,p1,p_v,  "p_E",ec,     I,C_f)
   
    plt.plot(x,yRNDD, label="RPU Varying p_E")
    plt.xlabel('p_E')
    plt.ylabel('Util') 
    p_Einflection = p_EInflection(n,PR,p1,pi,p_v,C_f,ec)
    
    print("p_E:\t"+str(p_Einflection))

    plt.legend()
    plt.show

In [22]:
def p_EInflection(n,PR,p1,pi,p_v,C_f,ec):
    
    D =   pi * PR + (pi* n)
    p2=1-p1
    
    
    p_EInflectionTop = -pi*p_v*p1
    p_EInflectionTop += pi*(p_v-1)
    p_EInflectionTop += pi*p_v*p2*(p1**n - 1)
    p_EInflectionTop += p_v*p2*(p1**n) *D
    p_EInflectionTop -= p_v*(p1**n -1) *(pi-C_f)
    p_EInflectionTop -= p_v*(p1**n) *(C_f+D)
    p_EInflectionTop -= (p_v-1)*(pi-C_f)
    
    p_EInflectionBottom = -2*(p_v*p1*ec)
    
    
  
    p_EInflectionBottom += (p_v-1)*2*ec
    p_EInflectionBottom += p_v*p2*(p1**n - 1)*2*ec
    p_EInflectionBottom += p_v*p2*(p1**n) *2*ec
 
    p_EInflectionFinal=p_EInflectionTop/p_EInflectionBottom

  
    return p_EInflectionFinal

In [23]:
#pi,B, computation, fine, cj, cd, m):
wn=widgets.IntSlider(min=0,max=10,step=1,value=4, description='n', continuous_update=False)
wPR=widgets.FloatSlider(min=0,max=100,step=0.01,value=4, continuous_update=False)
wNDPR=widgets.FloatSlider(min=0,max=100,step=0.01,value=4, continuous_update=False)

wNDRPR=widgets.IntSlider(min=0,max=100,step=0.01,value=4, continuous_update=False)
wpi=widgets.IntSlider(min=0,max=100,step=1,value=4, continuous_update=False)
wB=widgets.IntSlider(min=0,max=100,step=1,value=5, continuous_update=False)
wp_v = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.5, continuous_update=False)
wC_v=widgets.IntSlider(min=0,max=100,step=1,value=2, continuous_update=False)
wroi=widgets.FloatSlider(min=0,max=1,step=0.01,value=0.5, continuous_update=False)
wInsurance=widgets.FloatSlider(min=0,max=1,step=0.01,value=.5, continuous_update=False)
wC_f=widgets.FloatSlider(min=0,max=100,step=1,value=4, continuous_update=False)
wp1=widgets.FloatSlider(min=0,max=1,step=0.01,value=0.1, continuous_update=False)



#wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='P(run)', continuous_update=False) # probability that RP runs correctly



interactive_plotRP = widgets.interactive(plotRP,n=wn,PR=wPR, NDPR=wNDPR, p1=wp1, pi=wpi, b=wB, p_v=wp_v, C_v=wC_v,roi=wroi, Insurance=wInsurance, C_f=wC_f)
output = interactive_plotRP.children[-1]
output.layout.height = '600px'
interactive_plotRP

interactive(children=(IntSlider(value=4, continuous_update=False, description='n', max=10), FloatSlider(value=…

In [24]:
def RPp_EFinder(n,PR,p1,pi,p_v,C_f,ec):
    p2=1-p1
    if p_v==0: #
        if (C_f  >ec ):
            return 1
        elif (C_f  < ec ):
            return 0
        
    fraudCost=C_f
    execCost=ec
    D=pi * PR + (pi * n)
        
    #val finds if p_E0 or p_E1 is higher
    val = p_v*p1*(pi-execCost)
    val +=p_v*p2*(1-p1**n)*(pi-execCost)
    val +=p_v*p2*(p1**n)*(-D-execCost)
    val +=(1-p_v) * (fraudCost - execCost)
    val -=p_v*(1-p1**n)*(pi-fraudCost)
    val -= p_v*(p1**n)*(-D-fraudCost)
    
    if val>=0:
        p_E=1
    elif val<0:
        p_E=0
   
   
    
    return p_E
    
   

In [25]:
test = RPp_EFinder(1,1,1,2,3,4,5)

In [26]:
def RPNDoptimal(n,PR,NDPR,p1,pi,p_v,C_f,ec,I):
    p_Enew = RPp_EFinder(n,PR,p1,pi,p_v,C_f,ec)
    
    # Temporary fix
#     ec = pi*(1-roi)
#     D = pi * PR + (pi * n)
#     I = D*I
    
    RPutil = RPU(n,PR,NDPR,p1,pi,p_v,C_f,p_Enew, ec, I)
    return(RPutil)

In [27]:
def plotnonDetFINAL(n,PR,NDPR,pi,b, p_ETarget,p_v=1,C_v=0, p_E=1,roi=1,Insurance=0,C_f=0):   
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,101,retstep=True)
    print("step size: %s" %step)
    yJND = list(map(lambda p1: JCU(n,PR,NDPR, p1,b,pi,p_v=p_v,C_v=C_v, p_E=p_E),x))
    #yRND = list(map(lambda p1: RPNDU(n,PR,NDPR, p1,pi,p_v=p_v, p_E=p_E,roi=roi,I=Insurance),x))
    
    ec = pi*(1-roi)
    D = pi * PR + (pi * n)
    I = D*Insurance
    
    yRNDD = list(map(lambda p1: RPU(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    yRNDD0 = list(map(lambda p1: RPU(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,p_E=0,ec=ec,I=I),x))
    yRNDD1 = list(map(lambda p1: RPU(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,p_E=1,ec=ec,I=I),x))
    
    yRNDoptimal =list(map(lambda p1: RPNDoptimal(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,ec=ec,I=I),x))
    
    #                        RPp_EFinder(n, PR, p1, pi, p_v,   C_f,     roi):
    yp_E = list(map(lambda p1: RPp_EFinder(n, PR, p1, pi, p_v=p_v, C_f=C_f, ec=ec), x))
    
    #plt.plot(x,yJND, label="JUND")
    #plt.plot(x,yRND, label="RUND")
    plt.plot(x,yRNDD, label="RUNDD")
    
    plt.plot(x,yp_E, label="p_E Cheat Amount")
    plt.plot(x,yRNDD0, label="RUNDD, p_E=0 CHEATING")
    plt.plot(x,yRNDD1, label="RUNDD, p_E=1 FAIR")
    plt.plot(x,yRNDoptimal, label="RUNDoptimal")
    
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("n: %s" %n)
    print("Price: %s" %pi)
    D = pi * PR + pi*n
    print("D: %s" %D)
    damages = pi + C_r
    
    maxJND = max(yJND)
    p1Max = x[yJND.index(maxJND)]
    p2 = 1 - p1Max
    print("P(a1) for JCmax/RPmin util: %s" %p1Max)
    print("")
    
    print("Resource Provider")
    execCost = pi*(1-roi)
    requstedPrice = execCost + D*Insurance   
    print("requstedPrice: %s" %requstedPrice)   
    print("pi > requstedPrice, Matchable? : %s" %(pi>requstedPrice))
    
    print("")
    P1payout = p_v*p1Max*pi
    P2payout = p_v*p2*(1-p1Max**n)*damages
    TPayout = P1payout+P2payout
    RPcost = pi*(1-roi) 
    NDI = D*Insurance
    NDloss = p_v*(p2)*(p1Max**n)*D
    Tcost = RPcost + NDloss - NDI
    I = p_v*(p2)*(p1Max**n)
    
    p1=p_ETarget
    p2=1-p1 #1-p1
    
    fraudcost=C_f
    fraudCost=C_f
    
    p_Epoint =  - (fraudcost - execCost)/(p1*(pi-execCost)+p2*(1-p1**n)*(pi-execCost)+p2*(p1**n)*(-D-execCost)-(1-p1**n)*(pi-fraudCost)- (p1**n)*(-D-fraudCost) )

    
    optimalp_E = RPp_EFinder(n,PR,p1,pi,p_v,C_f,roi)

    print("p_E INFLECTION POINT: (P1 = "+str(p_ETarget)+"):\t necessary P = "+str(p_Epoint))
    
   
    print("Optimal p_E: (P1 = "+str(p_ETarget)+"):\t  p_Eoptimal = "+str(optimalp_E))
    
    
    
    print("max ND JC utility: %s" %maxJND)    
    
    
    with open('RP_file.csv', mode='w') as RPfile:
        RPwriter = csv.writer(RPfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        RPwriter.writerow(['p1','RND0', 'RND1', 'RNDoptimal', 'p_Eoptimal'])
        for i in range(101):
            RPwriter.writerow([i/100-1/100, yRNDD0[i], yRNDD1[i], yRNDoptimal[i], yp_E[i] ])

   

In [28]:
# ui = widgets.HBox([wn,wPR,wNDPR,wpi,wp_v,wcj,wp_E,wroi,winsurance])
#wp_ETarget = widgets.FloatSlider(min=0,max=1,step=.01,value=1, description='p_E Inflection Target',continuous_update=False)#Non-deterministic Penalty rate

ui1 = widgets.HBox([wn,wPR,wNDPR])
ui2 = widgets.HBox([wB,wpi,wp_v,wC_v])
ui3 = widgets.HBox([wC_f,wp_E,wroi,winsurance])
ui4 = widgets.HBox([wp_ETarget])

# interactive_plot = widgets.interactive(plot, n=wn,PR=wPR,NDPR=wNDPR,pi=wpi,p_v=wp_v,cj=wcj, p_E=wp_E,roi=wroi,Insurance=winsurance);
interactive_plotFinal = widgets.interactive_output(plotnonDetFINAL,{'n':wn,'PR':wPR,'NDPR':wNDPR,'pi':wpi,'b':wB, 'p_ETarget':wp_ETarget, 'p_v':wp_v,'C_v':wC_v, 'p_E':wp_E,'roi':wroi,'Insurance':winsurance,'C_f':wC_f});

interactive_plotFinal.layout.height = '600px'

display(interactive_plotFinal,ui1,ui2,ui3,ui4)
# interactive_plotFinal


Output(layout=Layout(height='600px'))

# Derivative of JPNDU wrt p1
   Simplify: Following Equation must not be true for 0<=p1<1, and p1
   
   p_v*(B-pi-C_v)
   + p_v*p_E*(1-p1)
   + n *p_v*(p_E-1)*p1**(n - 1) *( (-(NDD + C_r + C_v)) - (C_v-r) ) 
   + (B-NDD-C_r-C_v)*p_v*p_E * (n * (1-p1)*p1**(n-1)  -  (1-p1**n)  )
   =0


In [29]:
def OptimalP(fraudcost,execCost, p1, pi, n, D):
    
    p2=1-p1
    fraudcost
    p1=n/(n+1)

    top = - (fraudcost - execCost)
    bot = p1*(pi-execCost) +p2*(1-p1**n)*(pi-execCost) +p2*(p1**n)*(-D-execCost) -(1-p1**n)*(pi-fraudcost) - (p1**n)*(-D-fraudcost)

    rightside=top/bot

    p_v=rightside/(1-rightside)
    
    p_v=p_v+0.001
    
    if (p_v<0):
        p_v=0
    elif (p_v>1):
        p_v=1

    return p_v
    
    

In [30]:
def pp_Efinder(n,PR,p1,pi,C_f,ec, D):
    execCost=ec
    fraudcost=C_f
    p_v=OptimalP(fraudcost,execCost, p1, pi, n, D)
  
    
    p_E=RPp_EFinder(n,PR,p1,pi,p_v,C_f,ec)

    
    return p_E

In [31]:
def pRPUtility(n, PR, p1, pi, fraudcost, execCost, D, roi, I):    
    p_v=OptimalP(fraudcost,execCost, p1, pi, n, D)

    p_E=pp_Efinder(n,PR,p1,pi,fraudcost, execCost, D)

    RPutility = RPU(n, PR, PR, p1, pi, p_v,fraudcost, p_E, execCost, I)
              # RPU(n,PR,NDPR,p1,pi,p_v=1,C_f=0,p_E=1, ec=0, I=0):
    RPutility0 = RPU(n, PR, PR, p1, pi, p_v,fraudcost, 0, execCost, I)    
    RPutility1 = RPU(n, PR, PR, p1, pi, p_v,fraudcost, 1, execCost, I)    
          
    #RPutility=-999
    return [RPutility,RPutility0, RPutility1]

In [32]:
def plotp(fraudcost,execCost, p1, pi, n, PR, D,roi, I):  
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,51,retstep=True)
    print("step size: %s" %step)
    
    yP =   list(map(lambda execCost: OptimalP(fraudcost,execCost, p1, pi, n, D),x))
    yPp_E =  list(map(lambda execCost: pp_Efinder(n,PR,p1,pi,fraudcost,execCost, D),x))
    #yPp_E1 = list(map(lambda execCost: RPp_EFinder(n,PR,p1,pi,1,fraudcost,execCost),x))
    #yPp_E0 = list(map(lambda execCost: RPp_EFinder(n,PR,p1,pi,0,fraudcost,execCost),x))
    yRPU = list(map(lambda execCost: pRPUtility(n,PR,p1,pi,fraudcost,execCost, D,roi,I),x))
                                    #pRPUtility(n, PR, p1, pi, fraudcost, execCost, D, roi, I):       
    #yJND = list(map(lambda p1: JCU(n,PR,NDPR, p1,b,pi,p_v=p_v,C_v=C_v, p_E=p_E),x))
#     yRND = list(map(lambda p1: RPNDU(n,PR,NDPR, p1,pi,p_v=p_v, p_E=p_E,roi=roi,I=Insurance),x))
    
    #ec = pi*(1-roi)
    #D = pi * PR + (pi * n)
    #I = D*Insurance

    #yRNDD = list(map(lambda p1: RPU(n,PR,NDPR,p1,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    
    #plt.plot(x,yPp_E0, label="Optimal p_E when p_v=0")
    #plt.plot(x,yPp_E1, label="Optimal p_E when p_v=1")
    plt.plot(x,yP, label="Optimal P")
    plt.plot(x,yPp_E, label="p_E based on optimal P")
    
    RP1 = np.array(yRPU)[:,2]
   
    RP0 = np.array(yRPU)[:,1]
    RPopt = np.array(yRPU)[:,0] 
    
    plt.plot(x,RPopt, label="RP Utility with Optimal P, Optimal p_E")
    plt.plot(x,RP0, label="RP Utility with Optimal P, p_E=0")
    plt.plot(x,RP1, label="RP Utility with Optimal P, p_E=1")
    #plt.plot(x,yJND, label="JUND")
    #plt.plot(x,yRND, label="RUND")
    #plt.plot(x,yRNDD, label="RUNDD")
    plt.xlabel('execution Cost')
    plt.ylabel('P (verification rate)') 
    
    plt.legend()
    plt.show   
   

In [33]:
# ui = widgets.HBox([wn,wPR,wNDPR,wpi,wp_v,wcj,wp_E,wroi,winsurance])

wFraudcost = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='fraudcost', continuous_update=False) #Penalty Rate
wexecCost = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='execCost', continuous_update=False) #Penalty Rate
wpi = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='pi', continuous_update=False) #Penalty Rate

wD = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='D', continuous_update=False) #Penalty Rate
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)


ui6 = widgets.HBox([wFraudcost, wpi, wn, wPR])
ui7 = widgets.HBox([wD, wroi, winsurance])

#plotp(fraudcost,execCost, p1, pi, n, D):  
interactive_plotp = widgets.interactive_output(plotp,{'fraudcost': wFraudcost, 'execCost':wComputation, 'p1':wp1, 'pi':wpi, 'n':wn, 'PR':wPR, 'D':wD, 'roi':wroi, 'I':winsurance});

#interactive_plot = widgets.interactive_output(plot,{'n':wn,'PR':wPR,'NDPR':wNDPR,'pi':wpi,'b':wB,'p_v':wp_v,'C_v':wC_v, 'p_E':wp_E,'roi':wroi,'Insurance':winsurance,'C_f':wC_f });


# output = interactive_plot.children[-1]
# output.layout.height = '900px'
display(interactive_plotp,ui6,ui7)

Output()